### Setup

In [40]:
import pandas as pd

##### Considerations

Orginal Dataset from Kaggle: https://www.kaggle.com/sobhanmoosavi/us-accidents

Since raw file weighted ~1.5 GB, we had to clean up the data (filter Start_Time only year 2019) and compress the CSV file to allow the csv to be pushed within GitHub's threshold.

###### Compression Information

compression_opts = dict(method='zip', archive_name='us_accidents_2019_zipped.csv')

us_accidents.to_csv('us_accidents_2019_zipped.csv', index=False, compression=compression_opts)

In [39]:
file_path = './us_accidents_2019_zipped.csv'
us_accidents = pd.read_csv(file_path, compression='zip')
us_accidents

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-879261,MapQuest,406.0,3,2019-12-26 17:09:52,2020-01-10 11:00:00,40.884850,-95.800362,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
1,A-879262,MapQuest,406.0,3,2019-12-26 17:10:32,2020-01-10 11:00:00,40.889641,-95.798630,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
2,A-982553,MapQuest,201.0,2,2019-12-25 09:49:34,2019-12-25 11:03:57,41.848064,-71.062386,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
3,A-982554,MapQuest,201.0,2,2019-12-25 12:52:38,2019-12-25 13:37:03,42.660690,-71.192589,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-982555,MapQuest,201.0,2,2019-12-25 20:43:26,2019-12-25 21:58:45,42.157021,-71.069061,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950969,A-4239402,Bing,NaN,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.99888,-117.37094,...,False,False,False,False,False,False,Day,Day,Day,Day
950970,A-4239403,Bing,NaN,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.76555,-117.15363,...,False,False,False,False,False,False,Day,Day,Day,Day
950971,A-4239404,Bing,NaN,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.77740,-117.85727,...,False,False,False,False,False,False,Day,Day,Day,Day
950972,A-4239405,Bing,NaN,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.98311,-118.39565,...,False,False,False,False,False,False,Day,Day,Day,Day
